Antaget dataformat:

Local Time | Open | High | Low | Close | Volume | Name

Dollars-bar funktionen skal indeholde:

* Tilføj ny 'Mean' kolonne, som er udregnes
$$
\begin{align*}
    \text{Mean_HL} = \frac{\text{High} + \text{Low}}{2}
\end{align*}
$$

* Tilføj variablen 'Dollarbar_cap', som er $\text{DollarBar_cap} = max(\text{High}) \cdot max(\text{volume})$ skal rundes op til nærmeste million

* Tilføj variablen 'Total_transaction' som er $\text{Total_transaction}=\text{Rest_transaction}$ udenfor loop, og $\text{Total_transaction}=\text{Total_transaction}+\text{Volume}$ inde i loop.

* Tilføj variablen 'Rest_volume', som er $\text{Rest_transaction} = \text{Total_transaction} - \text{Dollarbar_cap}$, med start værdi 0, og kun når 'Total_transaction' > DollarBar_cap

* Tilføj vairablen 'Mean_OC', som er $\text{Mean_OC} = \frac{\text{Open} + \text{Close}}{2}$

* Tilføj variablen 'High_list' and 'Low_list', som skal append 'High' og 'Low' og når Dollarbar_cap er nået, så tage max og min af listen




start den førstes variable 'open' til 'open' og sæt 'close til den sidste rækkes open og close middelværdi (Mean_OC[1])
start den næste 'open' med Mean_OC[1] og close med Mean_OC[2]




Gem variable

* 'Dato' skal være slut Dato
* 'Open' fra 'Open' i starten ellers 'Mean_OC'
* 'High' og 'Low' fra HL_list, hvor der er taget min og max
* 'Close' fra 'Mean_OC'
* 'Type' fra 'type' i for loop variable 
* 'Volume' som er Dollarbar_cap


Funktionen virker nok ikke helt optimalt i det special tilfælde, hvor vi skal lave en dollarsbar på den sidste observation i aktie data'et. Som funktionen er nu, så vil den sidste dollarsbar være "rest", og her kan vi lægge under Dollarsbar_cap, men også over, hvilket jeg ikke er sikker på om det er okay.

Index driller, men ved ikke om det er vigtigt.

In [1]:
import math


def roundup(x, k):
    return int(math.ceil(x / k)) * k

In [33]:
import matplotlib
import pandas as pd
import numpy as np
import time
import os as os


def dollarbar(df, unit):
    if isinstance(df, pd.DataFrame):
        
        ## Create empty dataframe
        df_Dollarbar = pd.DataFrame({'CPHTime' : [], 'Period' : [], 'Open' : [], 'Low' : [], 'High' : [], 
                                      'Close' : [], 'Volume' : [], 'Monetary_Volume' : [], 'Name' : [], 'Type' : []})
        
        for name in pd.unique(df['Name']):
    
            df_type = df[df['Name'] == name]
            df_type = df_type.reset_index(drop=True)
            Rest_transaction = 0
            Total_transaction = 0
            Type = df_type['Type'][0]
    
            ## Empty lists
            Time_list = []
            Open_list = []
            Low_list = []
            High_list = []
            Close_list = []
            Volume_list = []
            HL_list = []
            Period_list = []
    
            ## support vairables
            Volume_help_list = []
            iter_since_dollarbar = 0
    
            ## Dollarsbar cap
            max_High = np.max(df_type['High'])
            max_Volume = np.max(df_type['Volume'])
            Dollarbar_cap = roundup(max_High * max_Volume, 1000000000)
    
            ## First value in Open
            Open_list.append(df_type['Open'][0])
    
            for n in np.arange(df_type.shape[0]):        
            
                Time = df_type['CPHTime'][n]
                Open = df_type['Open'][n]
                High = df_type['High'][n]
                Low = df_type['Low'][n]
                Close = df_type['Close'][n]
                Volume = df_type['Volume'][n]
        
                Mean_HL = np.mean([High, Low])
                HL_list.append(High)
                HL_list.append(Low)
        
                Total_transaction = Total_transaction + Mean_HL*Volume 
                Mean_OC = np.mean([Open, Close])
        
                Volume_help_list.append(Volume)
        
                iter_since_dollarbar += 1
            
                if n % 10000 == 0:
                    print(n)
        
                if Dollarbar_cap < Total_transaction:   #and df_type.shape[0] > n
            
                    Rest_transaction = Total_transaction - Dollarbar_cap
            
                    max_High = np.max(HL_list)
                    min_Low = np.min(HL_list)
            
                    p = (Dollarbar_cap - Total_transaction + Mean_HL*Volume)/(Mean_HL*Volume) ## andel af volume som skal med i denne dollarsbar
                    Volume_help_list[-1] = Volume*p
                    Sum_volume = np.sum(Volume_help_list)
            
                    ## Saving dollarbar values
                    Time_list.append(Time)
                    Period_list.append(iter_since_dollarbar*unit)
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Close_list.append(Mean_OC)
                    Volume_list.append(Sum_volume)
                    Open_list.append(Mean_OC)
                    
                    HL_list = []
            
                    Volume_help_list = []
                    Volume_help_list.append(Volume*(1-p)) ## andel af volume som skal med i næste dollarsbar
                    
                    Total_transaction = Rest_transaction
            
                    iter_since_dollarbar = 0
            
                if df_type.shape[0] == n+1 and iter_since_dollarbar == 0:
                
                    Sum_volume = Volume * (1-p)
                
                    Close_list.append(Close)
                    Time_list.append(Time)  ## Time bliver det samme, hvilket ik er godt
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Volume_list.append(Sum_volume)
            
                    df_dummy = pd.DataFrame({'CPHTime' : Time_list, 'Period' : Period_list, 'Open' : Open_list, 'Low' : Low_list, 'High' : High_list, 
                                     'Close' : Close_list, 'Volume' : Volume_list, 
                                     'Monetary_Volume' : Dollarbar_cap, 'Name' : name, 'Type' : Type})
                    df_Dollarbar = pd.concat([df_Dollarbar,df_dummy.iloc[:-1,:]])
            
                if df_type.shape[0] == n+1 and iter_since_dollarbar > 0:
            
                    max_High = np.max(HL_list)
                    min_Low = np.min(HL_list)
            
                    Sum_volume = np.sum(Volume_help_list)
            
                    ## Saving dollarbar values
                    Time_list.append(Time)
                    Period_list.append(iter_since_dollarbar*unit)
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Close_list.append(Mean_HL)
                    Volume_list.append(Sum_volume)
            
                    df_dummy = pd.DataFrame({'CPHTime' : Time_list, 'Period' : Period_list, 'Open' : Open_list, 'Low' : Low_list, 'High' : High_list, 
                                     'Close' : Close_list, 'Volume' : Volume_list, 
                                     'Monetary_Volume' : Dollarbar_cap, 'Name' : name, 'Type' : Type})
                    df_dummy['Monetary_Volume'].iloc[-1] = Total_transaction
                    df_Dollarbar = pd.concat([df_Dollarbar,df_dummy])
        
        return df_Dollarbar
            
    else:
        print('Input need to be a Pandas DataFrame')
            




In [38]:

start=time.time()  # Start the time


## Read data

import os as os
os.chdir(r"C:\Users\Bruger\Documents\Google_Drev_data\SwissData")
df = pd.read_csv('SwissData2.txt')
#df = df.iloc[0:2000000,:]


## Running the function 
df_dollarbar = dollarbar(df, 1)


# Print the time usage:
end = time.time()  # Stop the time
print(f"Time used by LightGBM: {(end-start):.1f} seconds")

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

C:\Users\Bruger\AppData\Local\Temp\ipykernel_10116\3265479891.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dummy['Monetary_Volume'].iloc[-1] = Total_transaction


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
Time used by LightGBM: 327.7 seconds


In [43]:
df_dollarbar[df_dollarbar['Name'] == 'DAX']

,CPHTime,Period,Open,Low,High,Close,Volume,Monetary_Volume,Name,Type
0,2018-02-01 10:20:00,16.0,12926.2790,12925.280,12946.780,12934.2295,7.188102e+06,9.300000e+10,DAX,Index
1,2018-02-01 10:34:00,30.0,12934.2295,12910.279,12943.280,12918.8795,7.192245e+06,9.300000e+10,DAX,Index
2,2018-02-01 10:54:00,50.0,12918.8795,12906.280,12925.780,12911.7295,7.199144e+06,9.300000e+10,DAX,Index
3,2018-02-01 11:03:00,59.0,12911.7295,12887.779,12922.480,12891.7300,7.204908e+06,9.300000e+10,DAX,Index
4,2018-02-01 11:08:00,64.0,12891.7300,12847.979,12894.980,12855.8795,7.224426e+06,9.300000e+10,DAX,Index
...,...,...,...,...,...,...,...,...,...,...
10310,2022-04-29 18:54:00,1393563.0,14047.7780,14043.749,14145.269,14141.7590,6.596340e+06,9.300000e+10,DAX,Index
10311,2022-04-29 20:13:00,1393642.0,14141.7590,14082.749,14191.399,14171.7840,6.583224e+06,9.300000e+10,DAX,Index
10312,2022-04-29 22:01:00,1393750.0,14171.7840,14048.249,14181.399,14081.6740,6.599450e+06,9.300000e+10,DAX,Index
10313,2022-04-30 01:31:00,1393960.0,14081.6740,13947.649,14083.669,13950.9380,6.630220e+06,9.300000e+10,DAX,Index


### Saving data

In [ ]:
df_dollarbar.to_csv('DollarbarData.txt', index = False, header=True)